# find git repo
[here](https://github.com/despresj/ece_colab)

In [1]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-8f026115-6af0-ddd1-bc81-466754818cee)


In [2]:

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [3]:

if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  proj_dir = "/content/drive/MyDrive/ece884_project/"
else:
  proj_dir = "../"

Mounted at /content/drive


In [4]:

df = pd.read_csv(f"{proj_dir}data_clean/clean.csv")
column_names = df.columns
df = df.to_numpy()

In [5]:

from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(df)
df = scaler.transform(df)

In [6]:

generator = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(200, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(400, activation="selu"),

    keras.layers.BatchNormalization(),
    keras.layers.Dense(400, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(200, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(98, activation="sigmoid")
])

In [7]:

discriminator = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(200, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(400, activation="selu"),
    keras.layers.BatchNormalization(),

    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(200, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1, activation="sigmoid") # not correct output dim
])

In [8]:
gan = keras.models.Sequential([generator, discriminator])

discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [9]:
codings_size = 98
batch_size = 32 # number of data points to return
dataset = tf.data.Dataset.from_tensor_slices(df).shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

In [10]:

from tqdm import tqdm

In [11]:

# Although this runs it still may have bugs
def train_gan(gan, dataset, batch_size, codings_size, iterations=1000, n_noise=20000):
    generator, discriminator = gan.layers
    auc_log = []
    epoch = 0
    auc = 0
    m = tf.keras.metrics.AUC(num_thresholds=3)
    #while auc < 1:
    for _ in range(500):
        epoch += 1
        print(f"Epoch {epoch}")
        for _ in tqdm(range(iterations)):
            random_index = np.random.permutation(len(df))
            X_batch = df[random_index, :]
            # phase 1 - training the discriminator
            noise = tf.random.normal(shape=(n_noise,98))
            generated_data = generator(noise)
            X_fake_and_real = tf.concat([generated_data, X_batch], axis=0)

            y1 = tf.constant([[0.]] * noise.shape[0] + [[1.]] * X_batch.shape[0])
            y1 = np.reshape(y1, (len(y1), 1))

            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)
            # phase 2 - training the generator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)
                    
        yhat = discriminator.predict(X_fake_and_real)

        m.update_state(y1, yhat)
        auc = m.result().numpy()

        print(auc)
        if epoch % 5 == 0:
            generator.save(f"{proj_dir}saved_models/gan{last_model+1}")

In [ ]:

import os
import re
gans_saved = os.listdir(f"{proj_dir}saved_models/")
model_number = [int(re.sub("gan", "", x)) for x in gans_saved]
last_model = max(model_number)

train_gan(gan, df, batch_size, codings_size, iterations=10000)

Epoch 1


100%|██████████| 10000/10000 [15:35<00:00, 10.69it/s]


1.0
Epoch 2


100%|██████████| 10000/10000 [15:22<00:00, 10.84it/s]


1.0
Epoch 3


...

100%|██████████| 10000/10000 [16:03<00:00, 10.38it/s]


1.0
INFO:tensorflow:Assets written to: /content/drive/MyDrive/ece884_project/saved_models/gan19/assets
Epoch 91


100%|██████████| 10000/10000 [16:01<00:00, 10.40it/s]


1.0
Epoch 92


 51%|█████     | 5073/10000 [08:15<08:03, 10.19it/s]

In [ ]:
generator.save(f"{proj_dir}saved_models/gan{last_model+1}")

# Need to plot this

In [ ]:
noise = tf.random.normal(shape=[batch_size, codings_size])
generated_data = generator(noise)
generated_data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.DataFrame(df, columns=column_names)
generated_array = generated_data[0, :]
generated_df = pd.DataFrame(np.reshape(generated_array, (1, 98)), columns=column_names)

In [ ]:
generated_df.loc[:,"hospnum"]

In [ ]:
fig, ax = plt.subplots(7, 14, figsize=(50,50))
ax = ax.ravel()

for i, column in enumerate(df.columns):
    df[column].hist(ax=ax[i], color="skyblue")
    # generated_df[:,column].hist(ax=ax[i], color="red")
    

In [ ]:
fig, ax = plt.subplots(7, 14, figsize=(50,50))
ax = ax.ravel()

for i, column in enumerate(df.columns):
    generated_df[column].hist(ax=ax[i], color="skyblue")